<a href="https://colab.research.google.com/github/johanjun/Google_Machine_Learning_Bootcamp_KR_2021/blob/main/kaggle/RSNA_MICCAI_BRAIN_TUMOR/rsna_miccai_brain_tumor_johanjun_1006_data_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 5.2 MB/s 


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm
import random
import pydicom
import cv2
import pickle

import matplotlib.pyplot as plt
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = '/content/drive/MyDrive/Google_Machine_Learning_Bootcamp_KR_2021/RSNA_MICCAI/data/rsna-miccai/'

In [ ]:
train_df = pd.read_csv(data_dir + "train_labels.csv")
test_df = pd.read_csv(data_dir + "sample_submission.csv")

In this discussion a competition host has notified that there are some issues with these 3 cases
Patient IDs -

1. 00109 (FLAIR images are blank)
2. 00123 (T1w images are blank)
3. 00709 (FLAIR images are blank)
<br><br>Hence these can be excluded

In [ ]:
#refer: https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data
EXCLUDE = [109, 123, 709]
train_df = train_df[~train_df.BraTS21ID.isin(EXCLUDE)]

In [ ]:
TYPES = ["FLAIR", "T1w", "T1wCE", "T2w"] #mpMRI scans

In [ ]:
def load_dicom(path, size = 64): #load DICOM files
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array #returns a numpy.ndarray containing the pixel data
    if np.max(data) != 0:
        data = data / np.max(data) #standardizes so that the pixel values are between 0 and 1
    data = (data * 255).astype(np.uint8) #rescales to 0 and 255
    return cv2.resize(data, (size, size))

In [ ]:
def get_all_image_paths(BraTS21ID, image_type, folder="train"): #get an array of all the images of a particular type or a particular patient id
    assert(image_type in TYPES) #only in types
    patient_path = os.path.join(data_dir + "%s/" % folder, str(BraTS21ID).zfill(5)) #다른 폴더일 수도 있음
    #print(lambda x: int(x[:-4].split("-")[-1]))
    
    paths = sorted(glob.glob(os.path.join(patient_path, image_type, "*")), key=lambda x: int(x[:-4].split("-")[-1])) #sort
    #print(paths)
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)
    if num_images < 10:
        jump = 1
    else:
        jump = 3
        
    return np.array(paths[start:end:jump])

In [ ]:
def get_all_images(BraTS21ID, image_type, folder="train", size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(BraTS21ID, image_type, folder)]

In [ ]:
def get_all_data_train(image_type):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        tmp_x = train_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "train", IMAGE_SIZE)
        label = tmp_x["MGMT_value"]

        X += images
        y += [label] * len(images)
        train_ids += [int(tmp_x["BraTS21ID"])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

def get_all_data_test(image_type):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        tmp_x = test_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "test", IMAGE_SIZE)
        X += images
        test_ids += [int(tmp_x["BraTS21ID"])] * len(images)

    return np.array(X), np.array(test_ids)

In [ ]:
#pickling
with open(data_dir+'X_T1wCE.pkl', 'rb') as f:
  X_T1wCE = pickle.load(f)

with open(data_dir+'y_T1wCE.pkl', 'rb') as f:
  y_T1wCE = pickle.load(f)

with open(data_dir+'train_idt_T1wCE.pkl', 'rb') as f:
  train_idt_T1wCE = pickle.load(f)

with open(data_dir+'X_test_T1wCE.pkl', 'rb') as f:
  X_test_T1wCE = pickle.load(f)

with open(data_dir+'test_idt_T1wCE.pkl', 'rb') as f:
  test_idt_T1wCE = pickle.load(f)
#pickling
with open(data_dir+'X_FLAIR.pkl', 'rb') as f:
  X_FLAIR = pickle.load(f)

with open(data_dir+'y_FLAIR.pkl', 'rb') as f:
  y_FLAIR = pickle.load(f)

with open(data_dir+'train_idt_FLAIR.pkl', 'rb') as f:
  train_idt_FLAIR = pickle.load(f)

with open(data_dir+'X_test_FLAIR.pkl', 'rb') as f:
  X_test_FLAIR = pickle.load(f)

with open(data_dir+'test_idt_FLAIR.pkl', 'rb') as f:
  test_idt_FLAIR = pickle.load(f)
#pickling
with open(data_dir+'X_T2w.pkl', 'rb') as f:
  X_T2w = pickle.load(f)

with open(data_dir+'y_T2w.pkl', 'rb') as f:
  y_T2w = pickle.load(f)

with open(data_dir+'train_idt_T2w.pkl', 'rb') as f:
  train_idt_T2w = pickle.load(f)

with open(data_dir+'X_test_T2w.pkl', 'rb') as f:
  X_test_T2w = pickle.load(f)

with open(data_dir+'test_idt_T2w.pkl', 'rb') as f:
  test_idt_T2w = pickle.load(f)
#pickling
with open(data_dir+'X_T1w.pkl', 'rb') as f:
  X_T1w = pickle.load(f)

with open(data_dir+'y_T1w.pkl', 'rb') as f:
  y_T1w = pickle.load(f)

with open(data_dir+'train_idt_T1w.pkl', 'rb') as f:
  train_idt_T1w = pickle.load(f)

with open(data_dir+'X_test_T1w.pkl', 'rb') as f:
  X_test_T1w = pickle.load(f)

with open(data_dir+'test_idt_T1w.pkl', 'rb') as f:
  test_idt_T1w = pickle.load(f)

In [ ]:
# %load train_lib.py

def train():
    import wandb
    config_defaults = {
        'MRI': 'FLAIR',
        'test_size' :  0.05,
        'N_EPOCHS' : 20,
        'N_BATCH' : 32,

    }
    wandb.init(project='RSNA_1006_datavalidation',
               config=config_defaults,
               magic=True)

    config = wandb.config
    
    MRI_type = config.MRI

    if MRI_type == 'FLAIR':
      X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_FLAIR, y_FLAIR, train_idt_FLAIR, test_size=config.test_size)
    elif MRI_type == 'T1wCE':
      X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_T1wCE, y_T1wCE, train_idt_T1wCE, test_size=config.test_size)
    elif MRI_type == 'T1w':
      X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_T1w, y_T1w, train_idt_T1w, test_size=config.test_size)
    elif MRI_type == 'T2w':
      X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X_T2w, y_T2w, train_idt_T2w, test_size=config.test_size)

    y_train = to_categorical(y_train)
    y_valid = to_categorical(y_valid)

    X_train = np.expand_dims(X_train, axis=-1)
    X_valid = np.expand_dims(X_valid, axis=-1)

    X2 = np.concatenate([X_FLAIR, X_T2w, X_T1w, X_T1wCE], axis=0)
    y2 = np.concatenate([y_FLAIR, y_T2w, y_T1w, y_T1wCE], axis=0)
    train_idt2 = np.concatenate([train_idt_FLAIR, train_idt_T2w, train_idt_T1w, train_idt_T1wCE], axis=0)

    X_train2, X_valid2, y_train2, y_valid2, train_idt_train2, train_idt_valid2 = train_test_split(X2, y2, train_idt2, test_size=0.2)

    X_train2 = np.expand_dims(X_train2, axis=-1) 
    X_valid2 = np.expand_dims(X_valid2, axis=-1)

    y_train2 = to_categorical(y_train2) 
    y_valid2 = to_categorical(y_valid2)

    IMAGE_SIZE = 64
    N_EPOCHS = config.N_EPOCHS
    N_BATCH = config.N_BATCH
    N_TRAIN = X_train.shape[0]
    N_VAL = X_valid.shape[0]
    steps_per_epoch = N_TRAIN / N_BATCH
    validation_steps = int(np.ceil(N_VAL / N_BATCH))

    def create_model():
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='VALID', 
                                    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      
      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='VALID'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      model.add(tf.keras.layers.Dropout(0.3))

      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
      model.add(tf.keras.layers.Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
      return model
    
    model = create_model()
    

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.AUC()])
    
    model.fit(
    x=X_train,
    y=y_train,
    epochs=N_EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_valid2, y_valid2))

In [ ]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'MRI': {
            'values': ['FLAIR', 'T1wCE', 'T1w', 'T2w']
        },
        'N_EPOCHS': {
            'values': [15, 50]
        },
        'N_BATCH': {
            'values': [32, 128]
        },
        'test_size': {
            'values': [0.05, 0.2]
        },
    }
}

In [ ]:
%pip install -q wandb

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 97 kB 6.0 MB/s 
     |████████████████████████████████| 180 kB 49.5 MB/s 
     |████████████████████████████████| 139 kB 52.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='RSNA_1006_datavalidation')

Create sweep with ID: amqwokwu
Sweep URL: https://wandb.ai/johanjunme/RSNA_1006_datavalidation/sweeps/amqwokwu


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: utqd5ut1 with config:
wandb: 	MRI: FLAIR
wandb: 	N_BATCH: 32
wandb: 	N_EPOCHS: 15
wandb: 	test_size: 0.05
wandb: Currently logged in as: johanjunme (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/15
369/369 [==============================] - 23s 14ms/step - loss: 2.2484 - auc: 0.6076 - val_loss: 1.0191 - val_auc: 0.5838
Epoch 2/15
369/369 [==============================] - 5s 12ms/step - loss: 0.8024 - auc: 0.6800 - val_loss: 0.7571 - val_auc: 0.5667
Epoch 3/15
369/369 [==============================] - 5s 12ms/step - loss: 0.6567 - auc: 0.7163 - val_loss: 0.7055 - val_auc: 0.5986
Epoch 4/15
369/369 [==============================] - 5s 13ms/step - loss: 0.6055 - auc: 0.7514 - val_loss: 0.6822 - val_auc: 0.6122
Epoch 5/15
369/369 [==============================] - 4s 12ms/step - loss: 0.5660 - auc: 0.7875 - val_loss: 0.7042 - val_auc: 0.6087
Epoch 6/15
369/369 [==============================] - 5s 12ms/step - loss: 0.5189 - auc: 0.8245 - val_loss: 0.7215 - val_auc: 0.6308
Epoch 7/15
369/369 [==============================] - 5s 12ms/step - loss: 0.4522 - auc: 0.8726 - val_loss: 0.7662 - val_auc: 0.6234
Epoch 8/15
369/369 [==============================] - 4s 12ms/step -

auc,▁▂▃▄▄▅▆▆▇▇█████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▃▃▃▂▂▂▂▂▂▁▁▁▁▁
val_auc,▂▁▃▃▃▄▄▅▆▆▆█▇▄▆
val_loss,▆▂▁▁▁▂▂▁▂▂▄▃▄█▆
auc,0.98935
best_epoch,3
best_val_loss,0.68222
epoch,14
loss,0.14532
val_auc,0.65506


wandb: Agent Starting Run: ratqmhfm with config:
wandb: 	MRI: FLAIR
wandb: 	N_BATCH: 32
wandb: 	N_EPOCHS: 15
wandb: 	test_size: 0.2
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

auc,▁▃▅▆▇▇██████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▁▁▅▆▆▅▇▆▇▇▇▇▇▇███▆▆▇█▇▇▆█▇▇▇▇▇▇█████████
val_loss,█▃▂▁▁▁▁▂▁▂▃▂▃▃▃▃▃▅▆▅▅▅▅▇▄▅▅▄▅▅▅▅▅▄▅▄▄▄▄▄
auc,1.0
best_epoch,7
best_val_loss,0.71715
epoch,49
loss,0.01026
val_auc,0.75769


wandb: Agent Starting Run: jgeu6hw7 with config:
wandb: 	MRI: T2w
wandb: 	N_BATCH: 128
wandb: 	N_EPOCHS: 50
wandb: 	test_size: 0.2
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/50
105/105 [==============================] - 5s 36ms/step - loss: 3.7749 - auc: 0.6226 - val_loss: 2.2696 - val_auc: 0.5757
Epoch 2/50
105/105 [==============================] - 3s 29ms/step - loss: 1.6184 - auc: 0.7407 - val_loss: 1.3157 - val_auc: 0.6149
Epoch 3/50
105/105 [==============================] - 3s 28ms/step - loss: 0.9762 - auc: 0.8108 - val_loss: 1.1404 - val_auc: 0.6293
Epoch 4/50
105/105 [==============================] - 3s 28ms/step - loss: 0.6790 - auc: 0.8705 - val_loss: 0.8114 - val_auc: 0.6913
Epoch 5/50
105/105 [==============================] - 3s 27ms/step - loss: 0.5047 - auc: 0.9140 - val_loss: 0.7787 - val_auc: 0.6763
Epoch 6/50
105/105 [==============================] - 3s 28ms/step - loss: 0.4203 - auc: 0.9307 - val_loss: 0.7892 - val_auc: 0.6976
Epoch 7/50
105/105 [==============================] - 3s 28ms/step - loss: 0.3868 - auc: 0.9364 - val_loss: 0.7195 - val_auc: 0.7298
Epoch 8/50
105/105 [==============================] - 4s 40ms/step - 

auc,▁▃▄▆▇▇▇█████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▁▃▃▆▆▇▄▇▇▇█▇████▇▇█▇▆▇█████▇█████████▇▇▆
val_loss,█▄▃▁▁▁▂▁▁▂▂▂▂▃▂▃▄▄▄▄▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▂▄
auc,0.99704
best_epoch,6
best_val_loss,0.71946
epoch,49
loss,0.08387
val_auc,0.69866


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
